<h1>Load the Dataset and Libraries</h1>

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForCausalLM

# Load the dataset
df = pd.read_csv('for_LLM_export.csv')

# Display the first few rows
df.head()

,SC_AGE_YEARS,sex_2122,allergies_2122,asthma_2122,headache_2122,anxiety_2122,depress_2122,behavior_2122,GeneticScr_2122,BrainInjTold_2122,...,ChHlthSt_2122,ExBrstFd_2122,DevDelay_2122,learning_2122,autism_2122,BedTime_2122,ACE1more4Com_2122,ACEincome_2122,ACE2more11_2122,ADHD_2122
0,5,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0
1,8,2,3.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,...,2.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0
2,5,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0
3,6,2,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,4.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0
4,17,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0


In [10]:
df1=df

In [11]:
df1.columns

Index(['SC_AGE_YEARS', 'sex_2122', 'allergies_2122', 'asthma_2122',
       'headache_2122', 'anxiety_2122', 'depress_2122', 'behavior_2122',
       'GeneticScr_2122', 'BrainInjTold_2122', 'ACE2more6HH_2122',
       'famstruct5_2122', 'fruit_2122', 'vegetables_2122', 'Cond2more_2122',
       'CSHCNtype_2122', 'ChHlthSt_2122', 'ExBrstFd_2122', 'DevDelay_2122',
       'learning_2122', 'autism_2122', 'BedTime_2122', 'ACE1more4Com_2122',
       'ACEincome_2122', 'ACE2more11_2122', 'ADHD_2122'],
      dtype='object')

<h1>Apply Mappings for Preprocessing</h1>

In [12]:
import pandas as pd

# Assume df1 is already loaded, e.g., via:
# df1 = pd.read_csv('your_file.csv')

# List of columns to convert to numeric (based on your mapping columns)
columns_to_convert = [
    'sex_2122', 'allergies_2122', 'asthma_2122', 'headache_2122', 'anxiety_2122', 
    'depress_2122', 'behavior_2122', 'GeneticScr_2122', 'BrainInjTold_2122', 'ACE2more6HH_2122',
    'famstruct5_2122', 'fruit_2122', 'vegetables_2122', 'Cond2more_2122', 'CSHCNtype_2122',
    'ChHlthSt_2122', 'ExBrstFd_2122', 'DevDelay_2122', 'learning_2122', 'autism_2122', 
    'BedTime_2122', 'ACE1more4Com_2122', 'ACEincome_2122', 'ACE2more11_2122', 'ADHD_2122'
]

# Convert these columns to numeric (non-numeric values become NaN)
for col in columns_to_convert:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')

# Provided mapping dictionary (maps descriptive string to integer)
mappings = {
    'sex_2122': {'Male': 1, 'Female': 2},
    'allergies_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'asthma_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'headache_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'anxiety_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'depress_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'behavior_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'GeneticScr_2122': {
        'Never Had Condition': 1, 
        'Ever Told, Not Identified By Test': 2, 
        'Ever Told, Identified By Test': 3
    },
    'BrainInjTold_2122': {
        'Never Thought Child Has Injury': 1, 
        'Ever Thought, Not Confirmed': 2, 
        'Ever Had Injury Confirmed By Doctor': 3
    },
    'ACE2more6HH_2122': {
        'No Aces': 1, 
        '1 Ace': 2, 
        '2 Or More Aces': 3
    },
    'famstruct5_2122': {
        'Two Parents, Married': 1, 
        'Two Parents, Not Married': 2, 
        'Single Parent': 3, 
        'Grandparent Household': 4, 
        'Other Family Type': 5
    },
    'fruit_2122': {
        'Never': 1, 
        'Rarely': 2, 
        'Sometimes': 3, 
        'Regularly': 4, 
        'Often': 5, 
        'Always': 6
    },
    'vegetables_2122': {
        'Never': 1, 
        'Rarely': 2, 
        'Sometimes': 3, 
        'Regularly': 4, 
        'Often': 5, 
        'Always': 6
    },
    'Cond2more_2122': {
        'None': 1, 
        'One Condition': 2, 
        'Multiple Conditions': 3
    },
    'CSHCNtype_2122': {
        'None': 0, 
        'Functional Limitations': 1, 
        'Prescription Medication Only': 2, 
        'Above-Routine Use Of Services': 3, 
        'Medication And Above-Routine Services': 4
    },
    'ChHlthSt_2122': {
        'Excellent': 1, 
        'Good': 2, 
        'Fair Or Poor': 3
    },
    'ExBrstFd_2122': {
        'Never': 1, 
        'Less Than 6 Months': 2, 
        '6 Months Regular Not Exclusively': 3, 
        '6 Months Exclusively': 4
    },
    'DevDelay_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'learning_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'autism_2122': {
        'Does Not Have Condition': 1, 
        'Ever Told, Does Not Currently Have': 2, 
        'Currently Has Condition': 3
    },
    'BedTime_2122': {
        'Always': 1, 
        'Usually': 2, 
        'Sometimes': 3, 
        'Rarely Or Never': 4
    },
    'ACE1more4Com_2122': {
        'No Aces': 1, 
        '1 Or More Aces': 2
    },
    'ACEincome_2122': {
        'No Aces': 1, 
        'Rarely': 2, 
        'Often': 3, 
        'Very Often': 4
    },
    'ACE2more11_2122': {
        'No Aces': 1, 
        'Single Ace': 2, 
        'Multiple Aces': 3
    },
    'ADHD_2122': {
        'Does Not Have ADHD': 0, 
        'Currently Has ADHD': 1
    },
}

# Invert each mapping so that integer codes become keys and descriptive strings become values.
inverted_mappings = {col: {v: k for k, v in mapping.items()} for col, mapping in mappings.items()}

# Apply the inverted mappings to the corresponding columns in df1
for col, inv_map in inverted_mappings.items():
    if col in df1.columns:
        df1[col] = df1[col].map(inv_map)

# Optionally, fill any unmapped or missing values with 'Unknown'
df1.fillna('Unknown', inplace=True)

# Display the first few rows of the decoded DataFrame
print(df1.head())


   SC_AGE_YEARS sex_2122                      allergies_2122  \
0             5     Male             Does Not Have Condition   
1             8   Female             Currently Has Condition   
2             5   Female             Does Not Have Condition   
3             6   Female  Ever Told, Does Not Currently Have   
4            17     Male             Does Not Have Condition   

               asthma_2122            headache_2122  \
0  Does Not Have Condition  Does Not Have Condition   
1  Currently Has Condition  Does Not Have Condition   
2  Does Not Have Condition  Does Not Have Condition   
3  Does Not Have Condition  Does Not Have Condition   
4  Does Not Have Condition  Does Not Have Condition   

                         anxiety_2122             depress_2122  \
0             Does Not Have Condition  Does Not Have Condition   
1  Ever Told, Does Not Currently Have  Does Not Have Condition   
2             Does Not Have Condition  Does Not Have Condition   
3             Does N

In [13]:
df1.head()

,SC_AGE_YEARS,sex_2122,allergies_2122,asthma_2122,headache_2122,anxiety_2122,depress_2122,behavior_2122,GeneticScr_2122,BrainInjTold_2122,...,ChHlthSt_2122,ExBrstFd_2122,DevDelay_2122,learning_2122,autism_2122,BedTime_2122,ACE1more4Com_2122,ACEincome_2122,ACE2more11_2122,ADHD_2122
0,5,Male,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Never Had Condition,Never Thought Child Has Injury,...,Excellent,Never,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Usually,No Aces,No Aces,No Aces,Does Not Have ADHD
1,8,Female,Currently Has Condition,Currently Has Condition,Does Not Have Condition,"Ever Told, Does Not Currently Have",Does Not Have Condition,Does Not Have Condition,Never Had Condition,Never Thought Child Has Injury,...,Good,6 Months Exclusively,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Always,No Aces,Rarely,Single Ace,Does Not Have ADHD
2,5,Female,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Never Had Condition,Never Thought Child Has Injury,...,Excellent,Never,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Usually,No Aces,No Aces,Single Ace,Does Not Have ADHD
3,6,Female,"Ever Told, Does Not Currently Have",Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Never Had Condition,Never Thought Child Has Injury,...,Excellent,6 Months Exclusively,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Usually,No Aces,Rarely,No Aces,Does Not Have ADHD
4,17,Male,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Never Had Condition,Never Thought Child Has Injury,...,Excellent,6 Months Exclusively,Does Not Have Condition,Does Not Have Condition,Does Not Have Condition,Always,No Aces,No Aces,Single Ace,Does Not Have ADHD


<h1> EDA</h1>

<h1>Train Test Split</h1>

In [14]:
from sklearn.model_selection import train_test_split

# Define a label mapping for ADHD
label_mapping = {
    "Does Not Have ADHD": 0,
    "Currently Has ADHD": 1
}

# Filter df1 to keep only rows with valid ADHD labels
df1 = df1[df1["ADHD_2122"].isin(label_mapping.keys())]

# Convert the target labels from strings to integers
y = df1["ADHD_2122"].map(label_mapping)

# Use all other columns (except ADHD_2122) as features
X = df1.drop("ADHD_2122", axis=1)

# For text-based input, combine all features into a single string per sample
X_text = X.astype(str).apply(lambda row: " ".join(row), axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)

print("Number of training samples:", len(X_train))
print("Number of test samples:", len(X_test))


Number of training samples: 53332
Number of test samples: 13334


<h1>Create a Custom Dataset Class</h1>

In [15]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx] if hasattr(self.labels, 'iloc') else self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



<h1>Tokenize the Data</h1>

In [17]:
from transformers import AutoTokenizer

# Choose a model checkpoint (this can be varied later, e.g., "bert-base-uncased", "distilbert-base-uncased", etc.)
model_checkpoint = "bert-base-uncased"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize the training and testing text data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)


<h1>Create a Custom PyTorch Dataset</h1>

In [67]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        # Ensure labels are stored as a list of integers
        self.labels = list(labels)
    
    def __getitem__(self, idx):
        # Convert each item in the encoding to a tensor
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Convert the label to a tensor (now an integer)
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)


<h1>Set Up the Model, Training Arguments, and Trainer</h1>

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

# Define a function to compute evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Load the pre-trained model (assuming a classification task with 2 labels)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

# Define the base directory for saving models, logs, and results
base_dir = r"E:\LLMS_ALL MODELS"

# Set up training arguments
training_args = TrainingArguments(
    output_dir=os.path.join(base_dir, 'results'),    # All models will be saved here
    num_train_epochs=3,                              # Number of training epochs
    per_device_train_batch_size=16,                  # Training batch size
    per_device_eval_batch_size=64,                   # Evaluation batch size
    evaluation_strategy="epoch",                     # Evaluate at the end of each epoch
    save_strategy="epoch",
    logging_dir=os.path.join(base_dir, 'logs'),      # Directory for logging
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

# Create PyTorch datasets from your encodings
train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\User\anaconda3\envs\llm_env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<h1>Loop Over Multiple Pre-trained Models for Training & Evaluation</h1>

In [20]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# Define the base directory for saving models, logs, and results
base_dir = r"E:\LLMS_ALL MODELS"

# Define a dictionary with model names and their corresponding checkpoints
model_dict = {
    "BERT": "bert-base-uncased",
    "DistilBERT": "distilbert-base-uncased",
    "ClinicalBERT": "emilyalsentzer/Bio_ClinicalBERT",
    "BioBERT": "dmis-lab/biobert-v1.1",
    "ALBERT": "albert-base-v2"
    # Add other models as needed
}

# Define a function to compute evaluation metrics: accuracy, precision, recall, F1, and AUC
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # Compute accuracy, precision, recall, and F1
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    
    # Compute probabilities using softmax for AUC calculation
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    prob_positive = probabilities[:, 1]  # probability of class '1'
    try:
        auc = roc_auc_score(labels, prob_positive)
    except ValueError:
        auc = 0.0
    
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

results_dict = {}

for model_name, model_checkpoint in model_dict.items():
    print(f"\nProcessing model: {model_name}")
    
    # 1. Load the tokenizer and tokenize the data
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)
    
    # Create datasets for the current model
    train_dataset = CustomDataset(train_encodings, y_train)
    test_dataset = CustomDataset(test_encodings, y_test)
    
    # 2. Load the pre-trained model for sequence classification (binary classification here: num_labels=2)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
    
    # 3. Set up training arguments with directories specific to this model
    training_args = TrainingArguments(
        output_dir=os.path.join(base_dir, 'results', model_name),
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir=os.path.join(base_dir, 'logs', model_name),
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="f1"
    )
    
    # 4. Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )
    
    # 5. Train the model and evaluate
    trainer.train()
    metrics = trainer.evaluate()
    print(f"Metrics for {model_name}: {metrics}")
    
    results_dict[model_name] = metrics

# Print final comparison of all models
print("\nFinal comparison of models:")
for model_name, metrics in results_dict.items():
    print(f"{model_name}: {metrics}")



Processing model: BERT


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.173000,0.247440,0.871456,0.915202,0.871456,0.886181,0.910916
2,0.292200,0.235159,0.891930,0.795540,0.891930,0.840982,0.817046
3,0.179000,0.237306,0.891930,0.795540,0.891930,0.840982,0.918995


C:\Users\User\anaconda3\envs\llm_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\User\anaconda3\envs\llm_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for BERT: {'eval_loss': 0.24744021892547607, 'eval_accuracy': 0.871456427178641, 'eval_precision': 0.9152022297681924, 'eval_recall': 0.871456427178641, 'eval_f1': 0.8861814585122536, 'eval_auc': 0.9109164629115745, 'eval_runtime': 36.4737, 'eval_samples_per_second': 365.578, 'eval_steps_per_second': 5.73, 'epoch': 3.0}

Processing model: DistilBERT


...

C:\Users\User\anaconda3\envs\llm_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for BioBERT: {'eval_loss': 0.3443581759929657, 'eval_accuracy': 0.891930403479826, 'eval_precision': 0.7955398446516853, 'eval_recall': 0.891930403479826, 'eval_f1': 0.8409821452083538, 'eval_auc': 0.7310684274592096, 'eval_runtime': 42.166, 'eval_samples_per_second': 316.227, 'eval_steps_per_second': 4.957, 'epoch': 3.0}

Processing model: ALBERT


C:\Users\User\anaconda3\envs\llm_env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--albert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and ar

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.187400,0.240086,0.871381,0.915181,0.871381,0.886124,0.905118
2,0.245100,0.224050,0.923879,0.917570,0.923879,0.919432,0.911040
3,0.208100,0.182868,0.927029,0.922087,0.927029,0.923764,0.950122


Metrics for ALBERT: {'eval_loss': 0.18286828696727753, 'eval_accuracy': 0.9270286485675716, 'eval_precision': 0.9220874055441293, 'eval_recall': 0.9270286485675716, 'eval_f1': 0.9237637642894898, 'eval_auc': 0.9501220196532663, 'eval_runtime': 46.2472, 'eval_samples_per_second': 288.32, 'eval_steps_per_second': 4.519, 'epoch': 3.0}

Final comparison of models:
BERT: {'eval_loss': 0.24744021892547607, 'eval_accuracy': 0.871456427178641, 'eval_precision': 0.9152022297681924, 'eval_recall': 0.871456427178641, 'eval_f1': 0.8861814585122536, 'eval_auc': 0.9109164629115745, 'eval_runtime': 36.4737, 'eval_samples_per_second': 365.578, 'eval_steps_per_second': 5.73, 'epoch': 3.0}
DistilBERT: {'eval_loss': 0.1422160267829895, 'eval_accuracy': 0.9344532773361331, 'eval_precision': 0.9322902790479016, 'eval_recall': 0.9344532773361331, 'eval_f1': 0.9332159079764323, 'eval_auc': 0.9683241087996468, 'eval_runtime': 19.2482, 'eval_samples_per_second': 692.74, 'eval_steps_per_second': 10.858, 'epoch'

<h1> Dataset related texts prediction </h1>

single

Multiple

<h1> AI GENERATED </h1>